In [46]:
import pandas as pd

df = pd.read_csv('agency.csv')
df

,AssetClass,AssetClassSubType,FNMATradeCount,FNMAUniqueID,FNMA$Trades,FHLMLCTradeCount,FHLMCUniqueID,FHLMLC$Trades,GNMATradeCount,GNMAUniqueID,GNMA$Trades,OtherTradeCount,OtherUniqueID,Other$Trades,Date
0,"AGENCY PASS-THRU (TBA, STIP, $ ROLLS)",SINGLE FAMILY 15Y,825,12,9911396.3,80,7,2116593.5,0,0,0,0,0,0,01/02/2019
1,"AGENCY PASS-THRU (TBA, STIP, $ ROLLS)",SINGLE FAMILY 30Y,5483,23,109532738.7,276,15,10163947.1,1140,28,19553447.6,0,0,0,01/02/2019
2,"AGENCY PASS-THRU (TBA, STIP, $ ROLLS)",OTHER,*,*,*,0,0,0,6,4,68920,0,0,0,01/02/2019
3,AGENCY PASS-THRU (SPECIFIED),SINGLE FAMILY 15Y,237,143,458682.1,155,113,680707.5,9,5,7895.9,0,0,0,01/02/2019
4,AGENCY PASS-THRU (SPECIFIED),SINGLE FAMILY 30Y,726,403,2122892.8,437,206,1498278.4,517,263,4305720.4,0,0,0,01/02/2019
5,AGENCY PASS-THRU (SPECIFIED),ADJUSTABLE/HYBRID,7,6,10662.1,*,*,*,24,19,55429.8,5,3,15838.4,01/02/2019
6,AGENCY PASS-THRU (SPECIFIED),OTHER,0,0,0,0,0,0,0,0,0,0,0,0,01/02/2019
7,AGENCY CMO,P&I,202,109,92506.7,249,131,194134,230,123,687908.6,8,8,3349.9,01/02/2019
8,AGENCY CMO,IO/PO,12,5,47308.3,*,*,*,*,*,*,*,*,*,01/02/2019
9,"AGENCY PASS-THRU (TBA, STIP, $ ROLLS)",SINGLE FAMILY 15Y,1632,12,16448879.8,135,9,1267312.2,0.0,0.0,0.0,0,0,0,01/03/2019


In [54]:
import pandas as pd

df = pd.read_excel('data/FINRA_IDS_PXTABLES-20190102.xlsx', sheet_name='TBA', skiprows=8, header=None)
df

,0,1,2,3,4,5,6,7,8
0,NaN,"PRICING TABLE: AGENCY PASS-THRU (TBA, STIP, $ ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,January Settlement,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Asset Sub-Class / Metric,COUPON,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,FNMA,<= 2.5,3,3.5,4,4.5,5,> 5.0
4,NaN,AVERAGE PRICE,97.8,99.9,101.3,102.4,0,0,0
5,NaN,WEIGHTED AVG. PRICE,97.8,99.9,101.2,102.4,0,0,0
6,NaN,AVG. PRICE BOTTOM 5 TRADES,97.7,99.8,101.2,102.3,0,0,0
7,NaN,2ND QUARTILE PRICE,97.8,99.8,101.2,102.3,0,0,0
8,NaN,3RD QUARTILE PRICE,97.8,99.9,101.3,102.4,0,0,0
9,NaN,4TH QUARTILE PRICE,97.8,99.9,101.3,102.4,0,0,0
